# Práctica 3: Desarrollo de una solución de minería de datos.

## Grupo: Javier Galiana Romero y Omar Moukket Rkizat

Como sabemos, la pandemia del COVID-19 ha sido, y es, uno de los hitos históricos más importante del mundo. En esta práctica, en la cual se busca el desarrollo de una solución de minería de datos sobre un problema real, se ha tomado la decisión de abordar justamente el tratamiento de la evolución de la vacunación contra la COVID-19 a nivel mundial.

Para ello se ha escogido un conjunto de datos proveniente de Kaggle que contiene los datos de vacunación diaria y total en cada país del mundo.

* Link de los datos: https://www.kaggle.com/gpreda/covid-world-vaccination-progress

Columnas del conjunto de datos:
- **`country`**: país al que petenece la información de vacunación.
- **`iso_code`**: código ISO para el país.
- **`date`**: fecha de los datos de entrada; para algunas fechas solo se tiene la vacunación diaria, y para otras solo la total (acumulativa).
- **`total_vaccinations`**: número total de vacunaciones en el país hasta la fecha indicada.
- **`people_vaccinated`**: en función de la estrategia de vacunación, una persona puede recibir una o dos dosis de vacunación. Por ello, en algún momento el número de vacunaciones será mayor que el número de personas vacunadas. Esta columna indica el número de personas vacunadas.
- **`people_fully_vaccinated`**: número total de personas que han recibido todas las dosis necesarias para la inmunización (normalmente dos vacunas).
- `daily_vaccinations_raw`: vacunaciones totales por día en el país y fecha indicados.
- **`daily_vaccinations`**: vacunaciones totales por día en el país y fecha indicados.
- **`total_vaccinations_per_hundred`**: ratio (porcentaje) entre el número de vacunaciones y el número de la población total del país hasta la fecha indicada. 
- **`people_vaccinated_per_hundred`**: ratio (porcentaje) entre el número de personas vacunadas y el número de la población total del país hasta la fecha indicada.
- **`people_fully_vaccinated_per_hundred`**: ratio (porcentaje) entre el número de personas totalmente inmunizadas y el número de la población total del país en la fecha indicada. 
-`daily_vaccinations_per_million`: ratio (ppm) entre el número de vacunaciones y el número de la población total del país en la fecha indicada. 
- **`vaccines`**: tipo de vacunas empleadas.
- `source_name`: fuente de la información (autoridad nacional, organización internacional, organización local, etc.).
- `source_website`: sitio web de la fuente de información.

## Pasos previos

En este apartado se importan las librerías necesarias para llevar a cabo el trabajo a lo largo de los diferentes apartados del notebook.

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import plotly.express as ex
from scipy import stats
from datetime import datetime
from keras.callbacks import EarlyStopping
from keras.constraints import Constraint
from keras.layers import Dense
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Lambda
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler

## Carga y preproceso de los datos

In [ ]:
# Se cargan los datos
data = pd.read_csv("../input/d/gpreda/covid-world-vaccination-progress/country_vaccinations.csv")

El conjunto de datos tiene un total de 15 columnas, de las cuales vamos a seleccionar solo las que nos interesan. Las columnas de interés son las que se han marcado en **negrita** en la explicación inicial del cuaderno.

In [ ]:
# Se declaran las columnas de interés
columns = ['iso_code', 'country', 'date', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 
           'daily_vaccinations', 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 
           'people_fully_vaccinated_per_hundred', 'vaccines']

# Se seleccionan las columnas y se guarda el nuevo dataframe
data = data[columns]

In [ ]:
# Vemos el aspecto de los datos
data.head()

Para enriquecer los datos y mejorar el proceso de análisis, vamos a cargar un segundo conjunto de datos que contienen las cifras de infecciones y muertes por COVID-19 en todos los países del mundo. El objetivo es juntar esta información con el primer conjunto de datos y sacar unas conclusiones más enrriquecedoras.

In [ ]:
# Se carga el segundo conjunto de datos
covid_data = pd.read_csv("../input/covid19global/WHO-COVID-19-global-data.csv")

In [ ]:
covid_data.head()

En primer lugar vamos a trasnformar la columna de fecha a tipo _datetime_ en ambos dataframes, ya que nos será de utilidad para acotar los datos de ambos conjuntos de datos a un mismo periodo. 

In [ ]:
covid_data['Date_reported'] = pd.to_datetime(covid_data['Date_reported'])

In [ ]:
# Resumen de los datos
covid_data.info()

In [ ]:
data['date'] = pd.to_datetime(data['date'])

Las columnas `total_vaccinations_per_hundred`,`people_vaccinated_per_hundred` y `people_fully_vaccinated_per_hundred` representan el porcentaje de vacunaciones, personas vacunadas y personas totalmente vacunadas de cada país en relación a su tamaño de población. Vamos a revisar que estos porcentajes estén en el rango [0,100] y en caso de no ser así se ajustarán a dicho rango.



In [ ]:
# Ejemplo de valores fuera de rango
data[data['total_vaccinations_per_hundred'] > 100]['total_vaccinations_per_hundred']

In [ ]:
# Clip outliers

MIN_VALUE = 0.
MAX_VALUE = 100.
data[['total_vaccinations_per_hundred',
      'people_vaccinated_per_hundred', 
      'people_fully_vaccinated_per_hundred']] = np.clip(data[['total_vaccinations_per_hundred',
                                                              'people_vaccinated_per_hundred', 
                                                              'people_fully_vaccinated_per_hundred']],
                                                        MIN_VALUE, 
                                                        MAX_VALUE)

## Estadística descriptiva

Este apartado va a contener todo el proceso de análisis exploratorio y la extracción de las conclusiones correspondientes.

Para empezar, vamos a comprobar qué proveedores o compañías de vacunas son las más empleadas y las que menos a nivel mundial. Para ello vamos a revisar la columna `vaccines` en la cual se indica qué vacunas son empleadas por cada país en cada fecha y contaremos las ocurrencias de cada una de ellas.  

In [ ]:
vaccines = data['vaccines'].value_counts()
vaccines

Como podemos ver en el conteo de arriba, hay casos en los se notifica una sola compañía y en otros casos un grupo de compañías. Por lo tanto, la solución va a ser dividir las instancias de grupos y sumar el conteo de cada compañía de manera individual. En la siguiente celda se realiza esta operación.

In [ ]:
# Diccionario que contendrá el conteo de cada compañía
vaccines_counter = {}

# Se itera por los valores y si son grupos se dividen y
# se gestiona el conteo acumulativo
for vaccine in vaccines.index:
    value_count = vaccines[vaccine]
    vaccine_split = vaccine.split(', ')
    for sub in vaccine_split:
        if sub in vaccines_counter.keys():
            vaccines_counter[sub] += value_count
        else:
            vaccines_counter[sub] = value_count

In [ ]:
# Vemos el resultado del conteo final
vaccines_counter

Para representar los resultados de manera gráfica vamos a representar un diagrama de árbol donde se vea la distribución de uso de las vacunas de cada proveedor a nivel mundial.

In [ ]:
plt.subplots(figsize=(10, 8))
squarify.plot(sizes=vaccines_counter.values(), label=vaccines_counter.keys(), pad=1, alpha=0.5)
plt.axis('off')  # Ocultamos los valores de los ejes
plt.title('Proveedores de vacunas empleados a nivel mundial')
plt.show()

En total hay 17 proveedores reflejados en nuestro conjunto de datos, pero como se puede ver en el gráfico, los que más destacan por su uso son 7: AstraZeneca, Pfizer, Moderna, Johnson&Johnson, Beijing, Sputnik V y Sinovac.

Siguiendo con el análisis, sería de gran interés conocer cómo se distribuye el porcentaje de población totalmente vacunada en los diferentes países. Primero vamos a ver cuál es la media y mediana de este porcentaje alrededor de todo el mundo.

In [ ]:
fully_vacc = data.groupby('country')['people_fully_vaccinated_per_hundred'].max().dropna()
print('Media del porcentaje de la población completamente inmunizada:', round(fully_vacc.mean(), 2))
print('Mediana del porcentaje de la población completamente inmunizada:', round(fully_vacc.median(), 2))

Como podemos ver, hay cierta diferencia entre la media y la mediana, lo cual significa que hay ciertos valores extremos que se salen de la distribución normal de los datos. Para conocer exactamente cómo es la distribución de estos datos vamos a calcular el coeficiente de asimetría (skewness) y observaremos el histograma correspondiente.

A continuación calculamos la media y la mediana del porcentaje de personas vacunadas (no totalmente inmunizadas) a nivel mundial

In [ ]:
partial_vacc = data.groupby('country')['people_vaccinated_per_hundred'].max().dropna()
print('Media del porcentaje de la población vacunada:', round(partial_vacc.mean(), 2))
print('Mediana del porcentaje de la población vacunada:', round(partial_vacc.median(), 2))

In [ ]:
asimetria_fully = stats.skew(fully_vacc)
asimetria_vacc = stats.skew(partial_vacc)
print('Coeficiente de asimetría:', round(asimetria_fully, 2))
print('Coeficiente de asimetría:', round(asimetria_vacc, 2))

Como regla general, un skewness menor que -1 o mayor que 1 indica que estamos ante una distribución extremadamente sesgada, y en este caso para la distribución de población totalmente inmunizada dicho valor es mayor que 1 y para la población parcialmente vacunada es menor que 1. A continuación se muestra el histograma para ambos casos y queda reflejado que tenemos una distribución altamente sesgada, con una asimetría positiva en el primer caso. Este resultado es de esperar, ya que cada país tiene su propia política de vacunación y por otro lado, no todos los países tienen la misma capacidad de acceso a las vacunas.

In [ ]:
plt.figure(figsize=(14,7))
sns.histplot(data=fully_vacc, kde=True)
plt.title('Porcentaje de población totalmente vacunada a nivel mundial')
plt.xlabel('% población inmunizada')
plt.show()

Por otro lado, para el segundo caso de vacunaciones parciales, la distribución no es tan extramadamente sesgada y muestra cierta simetría normal pero al mismo tiempo tiene una cola de asimetría positiva que justifica ese valor de skewness obtenido. Esta distribución es de esperar, ya que refleja cierta normalidad en el proceso de vacunación a nivel general, pero dejando también claro que hay un porcentaje de países que no están llevando el mismo ritmo de vacunación, siendo esto debido seguramente a la falta de capital económico. 

In [ ]:
plt.figure(figsize=(14,7))
sns.histplot(data=partial_vacc, kde=True)
plt.title('Porcentaje de población vacunada a nivel mundial')
plt.xlabel('% población vacunada')
plt.show()

A continuación vamos a calcular los cuartiles del porcentaje de población inmunizada y vacunada.

In [ ]:
# cuartiles población inmunizada
fully_vacc.quantile([0.25, 0.5, 0.75])

In [ ]:
plt.figure(figsize=(5, 5))
sns.set_theme(style="whitegrid")
sns.boxplot(data=fully_vacc, orient="v", palette="Set2")
plt.title('Cuartiles de porcentaje de inmunización')
plt.show()

Como podemos ver, el primer cuartil indica que el 25% de los países tienen tan solo un 1% o menos de la población completamente inmunizada. El segundo cuartil indica que el 50% de países tienen un 9% o menos de la población completamente inmunizada. Por último, el tercer cuartil indica que el 75% tienen alrededor de un cuarto de su población completamente inmunizada. Por otro lado, en el gráfico se pueden detectar algunos outliers, concretamente 2, de los cuales uno tiene más de un 60% de inmunizados y el otro un 100%. 



In [ ]:
# cuartiles población vacunada
partial_vacc.quantile([0.25, 0.5, 0.75])

In [ ]:
plt.figure(figsize=(5, 5))
sns.set_theme(style="whitegrid")
sns.boxplot(data=partial_vacc, orient="v", palette="Set2")
plt.title('Cuartiles de porcentaje de vacunación')
plt.show()

En el caso de la distribución de vacunaciones, el primer cuartil indica que el 25% de los países tienen tan solo un 3.5% o menos de la población vacunada. El segundo cuartil indica que el 50% de países tienen un 19% o menos de la población vacunada. Por último, el tercer cuartil indica que el 75% tienen alrededor de la mitad de su población vacunada. En este caso no se reflejan outliers.

Veamos ahora estas proporciones en un mapa mundial, donde la escala de color indica la proporción de población completamente vacunada. Como se puede ver en el resultado del gráfico, la mayoría de países están pintandos en escalas de color que representan menos de un 40% de la población completamente vacunada. Muy pocos casos están por encima de este umbral.

In [ ]:
fully_vacc = pd.DataFrame(fully_vacc)
fully_vacc.reset_index(level=0, inplace=True)

ex.choropleth(fully_vacc, locations="country", 
                    locationmode='country names',
                    color="people_fully_vaccinated_per_hundred", 
                    hover_name="country", 
                    title='Porcentaje de personas totalmente vacunadas a nivel mundial',
                    color_continuous_scale= ex.colors.sequential.Viridis,
                    width= 1000,
                    height= 400
                   )

Ahora vamos a centrarnos en cómo está evolucionando la vacunación en España y en ciertos países vecinos europeos. Para ello primero vamos a ver un gráfico de la serie temporal de vacunaciones por día al mismo tiempo que veremos cómo han ido evolucionando los casos de infección y muertes diarios durante las mismas fechas. Finalmente trataremos de implementar un modelo que nos permita predecir para España esta evolución fuera de las fechas que contiene nuestro conjunto de datos.

In [ ]:
# Países europeos de interés
countries = ['Spain', 'France', 'Germany', 'Portugal', 'United Kingdom']

In [ ]:
# Se unifica el nombre de UK
covid_data.loc[covid_data['Country'] == 'The United Kingdom', ['Country']] = 'United Kingdom'

A continuación vamos a sacar la información de cada uno de estos países y la vamos a guardar en una estructura conjunta

In [ ]:
europe_vacc = []
europe_covid = []

for country in countries:
    data_country = data[data['country'] == country]
    data_country.reset_index(inplace=True)
    europe_vacc.append(data_country)
    
    min_date = pd.to_datetime(data_country['date'].min(), format='%Y-%m-%d')
    max_date = pd.to_datetime(data_country['date'].max(), format='%Y-%m-%d')
    covid_data = covid_data[covid_data['Date_reported'] >= min_date] 
    covid_data = covid_data[covid_data['Date_reported'] <= max_date]
    covid_data_country = covid_data[covid_data['Country'] == country]
    europe_covid.append(covid_data_country)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(20, 20))
axes = [ax1, ax2, ax3, ax4, ax5]

sns.set_theme(style="whitegrid")

for i in range(len(axes)):
    data_vacc = europe_vacc[i]
    data_covid = europe_covid[i]
    ax = axes[i]
    
    ax.plot(data_vacc['date'], 
                 data_vacc['daily_vaccinations'], 
                 label='Vacunaciones')
    ax.plot(data_covid['Date_reported'], 
                 data_covid['New_cases'],
                 label='Infecciones')
    ax.plot(data_covid['Date_reported'], 
                 data_covid['New_deaths'],
                 label='Muertes')
    
    ax.legend(loc = 'upper left')
    ax.set_xlabel('Día')
    ax.set_ylabel('Vacunas')
    ax.set_title('Vacunación por día en {}'.format(countries[i]))
    
ax6.axis('off')
plt.show()

Como podemos ver, en países como España, Francia y UK los casos de infección se reducen de manera gradual a medida que se evoluciona en la vacunación. Sin embargo, en países como Alemania y Francia a pesar de aumentar el ritmo de vacunación se vuelven a generar algunos picos de infecciones durante el proceso.

A continuación vamos a ver el ritmo de vacunación a nivel mundial, enfocándonos en comparar los países de cada continente y luego compararemos el total de vacunaciones entre continentes, de manera que podamos identificar a los países más desfavorecidos ea nivel de continente y por otro lado a los continentes más desfavorecidos a nivel mundial.

In [ ]:
# Listas de códigos ISO de los países de cada continente

eu_countries = ['BEL', 'BGR', 'CZE', 'DNK', 'DEU', 'EST', 'IRL', 'GRC', 'ESP', 'FRA', 
                'HRV', 'ITA', 'CYP', 'LVA', 'LTU', 'LUX', 'HUN', 'MLT', 'NLD', 'AUT', 
                'POL', 'PRT', 'ROU', 'SVN', 'SVK', 'FIN', 'SWE']

af_countries = ['ZWE', 'DZA', 'AGO', 'BEN', 'BWA', 'BFA', 'BDI', 'CMR', 'CPV', 'CAF','TCD',
                'COM','COG','DJI','EGY','GNQ','ERI','ETH','GAB','GMB','GHA','GIN','GNB','CIV',
                'KEN','LSO','LBR','LBY','MDG','MWI','MLI','MRT','MUS','MYT','MAR','MOZ','NAM',
                'NER','NGA','REU','RWA','STP','SEN','SYC','SLE','SOM','ZAF','SSD','SHN','SDN',
                'SWZ','TGO','TUN','UGA','COD','ZMB','TZA']

as_countries = ['AFG','ARM','AZE','BHR','BGD','BTN','BRN','KHM','CHN','CXR','CCK','IOT','GEO',
                'HKG','IND','IDN','IRN','IRQ','ISR','JPN','JOR','KAZ','KWT','KGZ','LAO','LBN',
                'MAC','MYS','MDV','MNG','MMR','NPL','PRK','OMN','PAK','PSE','PHL','QAT','SAU',
                'SGP','KOR','LKA','SYR','TWN','TJK','THA','TUR','TKM','ARE','UZB','VNM','YEM']

na_countries = ['AIA','ATG','ABW','BHS','BRB','BLZ','BMU','VGB','CAN','CYM','CRI','CUB','CUW',
                'DMA','DOM','SLV','GRL','GRD','GLP','GTM','HTI','HND','JAM','MTQ','MEX','MSR',
                'ANT','NIC','PAN','PRI','BES','SXM','KNA','LCA','SPM','VCT','TTO','TCA','USA',
                'VIR']

sa_countries = ['ARG','BOL','BRA','CHL','COL','ECU','FLK','GUF','GUY','PRY','SUR','URY','VEN']

In [ ]:
data['people_vaccinated_per_hundred'] = data['people_vaccinated_per_hundred'].fillna(0)

# Se filtran los datos de los países de cada continente
data_eu = data.query('iso_code in @eu_countries')
data_af = data.query('iso_code in @af_countries')
data_as = data.query('iso_code in @as_countries')
data_na = data.query('iso_code in @na_countries')
data_sa = data.query('iso_code in @sa_countries')

In [ ]:
# Se extrae el máximo porcentaje de vacunación por país
data_eu = data_eu.groupby(['iso_code', 'country']).max('people_vaccinated_per_hundred').reset_index()
data_af = data_af.groupby(['iso_code', 'country']).max('people_vaccinated_per_hundred').reset_index()
data_as = data_as.groupby(['iso_code', 'country']).max('people_vaccinated_per_hundred').reset_index()
data_na = data_na.groupby(['iso_code', 'country']).max('people_vaccinated_per_hundred').reset_index()
data_sa = data_sa.groupby(['iso_code', 'country']).max('people_vaccinated_per_hundred').reset_index()

In [ ]:
data_sa.sort_values("people_vaccinated_per_hundred", ascending=False)

In [ ]:
ex.choropleth(data_eu, 
              locations='iso_code', 
              color='people_vaccinated_per_hundred',
              title='Vacunación en los paises de la Unión Europea',
#               width= 4000, 
#               height= 1000,
              hover_data=['country']).update_geos(scope="europe")
# plt.title('Vacunación en los paises de la Unión Europea')

In [ ]:
ex.choropleth(data_af, 
              locations='iso_code', 
              color='people_vaccinated_per_hundred', 
              title='Vacunación en los paises de África',
#               width= 4000,
#               height= 1000,
              hover_data=['country']).update_geos(scope="africa")

In [ ]:
ex.choropleth(data_as, 
              locations='iso_code', 
              color='people_vaccinated_per_hundred',
              title='Vacunación en los paises de Asia',
#               width= 4000,
#               height= 1000,
              hover_data=['country']).update_geos(scope="asia")

In [ ]:
ex.choropleth(data_na, 
              locations='iso_code', 
              color='people_vaccinated_per_hundred', 
              title='Vacunación en los paises América del Norte',
#               width= 4000,
#               height= 1000,
              hover_data=['country']).update_geos(scope="north america")

In [ ]:
ex.choropleth(data_sa, 
              locations='iso_code', 
              color='people_vaccinated_per_hundred', 
              title='Vacunación en los paises América del Sur',
#               width= 4000,
#               height= 1000,
              hover_data=['country']).update_geos(scope="south america")

In [ ]:
# Se extrae el máximo número de vacunación por país
data_eu = data_eu.groupby(['iso_code', 'country']).max('total_vaccinations').reset_index()
data_af = data_af.groupby(['iso_code', 'country']).max('total_vaccinations').reset_index()
data_as = data_as.groupby(['iso_code', 'country']).max('total_vaccinations').reset_index()
data_na = data_na.groupby(['iso_code', 'country']).max('total_vaccinations').reset_index()
data_sa = data_sa.groupby(['iso_code', 'country']).max('total_vaccinations').reset_index()

In [ ]:
# Vacunación acumulada por continente en proporción a su población
data_eu['total_vacc_continent'] = data_eu['total_vaccinations'].sum()/(446*(10**6))
data_af['total_vacc_continent'] = data_af['total_vaccinations'].sum()/(1216*(10**6))
data_as['total_vacc_continent'] = data_as['total_vaccinations'].sum()/(4561*(10**6))
data_na['total_vacc_continent'] = data_na['total_vaccinations'].sum()/(579*(10**6))
data_sa['total_vacc_continent'] = data_sa['total_vaccinations'].sum()/(422*(10**6))

In [ ]:
# Se juntan los datos en un único df
data_continent = data_eu.append(data_af).append(data_as).append(data_na).append(data_sa)

In [ ]:
ex.choropleth(data_continent, 
              locations='iso_code', 
              color='total_vacc_continent', 
              title='Vacunación por continentes',
#               width= 4000,
#               height= 1000,
              hover_data=['iso_code'])

## Modelo predictivo

### LSTM

In [ ]:
data_spain = data[data['country'] == 'Spain']

In [ ]:
data_spain.date.min()

In [ ]:
data_spain.date.max()

In [ ]:
TRAIN_START_DATE = '2021-01-04'
TRAIN_END_DATE = '2021-05-31'

TEST_START_DATE = '2021-06-01'
TEST_END_DATE = '2021-06-17'

NB_LOOKBACK_DAYS = 7
NUM_TRIALS = 5
LSTM_SIZE = 16

In [ ]:
b = pd.to_datetime(TRAIN_END_DATE, format='%Y-%m-%d')
a = pd.to_datetime(TRAIN_START_DATE, format='%Y-%m-%d')
delta = b - a 
NB_TRAIN_DAYS = delta.days + 1
print(NB_TRAIN_DAYS)

In [ ]:
b = pd.to_datetime(TEST_END_DATE, format='%Y-%m-%d')
a = pd.to_datetime(TEST_START_DATE, format='%Y-%m-%d')
delta = b - a
NB_TEST_DAYS = delta.days + 1
print(NB_TEST_DAYS)

In [ ]:
b = pd.to_datetime(TEST_END_DATE, format='%Y-%m-%d')
a = pd.to_datetime(TRAIN_START_DATE, format='%Y-%m-%d')
delta = b - a
TOTAL_DAYS = delta.days + 1
print(TOTAL_DAYS)

In [ ]:
def construct_model(nb_context, lstm_size=32, nb_lookback_days=14):

    # Create context encoder
    context_input = Input(shape=(nb_lookback_days, nb_context),
                          name='context_input')
    x = LSTM(lstm_size, name='context_lstm')(context_input)
    context_output = Dense(units=1, activation='softplus', 
                           name='context_dense')(x)

    model = Model(inputs=[context_input],
                  outputs=[context_output])
    model.compile(loss='mae', optimizer='adam')

    # Create training model, which includes loss to measure
    # variance of action_output predictions
    training_model = Model(inputs=[context_input],
                           outputs=[context_output])
    training_model.compile(loss='mae',
                           optimizer='adam')

    return model, training_model

def create_country_samples(df):
    # Variable a predecir
    context_column = 'daily_vaccinations'
    outcome_column = 'daily_vaccinations'
    # Inicialización del diccionario
    country_samples = {}

    cdf = df.copy()
    # Se filtran las instancias no nulas
    cdf = cdf[cdf['daily_vaccinations'].notnull()]
    # Se extrae la columna que se usa para el entrenamiento
    context_data = np.array(cdf[context_column])
    context_samples = []

    # Se extrae la columna correspondiente a la variable de salida
    outcome_data = np.array(cdf[outcome_column])
    outcome_samples = []
    # Total de días para el entrenamiento
    nb_total_days = min(context_data.shape[0], NB_TRAIN_DAYS)

    for d in range(NB_LOOKBACK_DAYS, nb_total_days):
        # Se van cogiendo lotes de NB_DAYS días en forma de ventana deslizante
        context_samples.append(context_data[d - NB_LOOKBACK_DAYS:d])
        outcome_samples.append(outcome_data[d])

    if len(context_samples) > 0:
        X_context = np.expand_dims(np.stack(context_samples, axis=0), axis=2)
        # Se rellena el diccionario con los datos de entrada (X)
        country_samples = {
            'X_context': X_context,
            'X_train_context': X_context[:-NB_TEST_DAYS],
            'X_test_context': X_context[-NB_TEST_DAYS:],
        }

        # Se rellena el diccionario con los datos de salida (Y)
        y = np.stack(outcome_samples, axis=0)
        country_samples['y'] = y
        country_samples['y_train'] = y[:-NB_TEST_DAYS]
        country_samples['y_test'] = y[-NB_TEST_DAYS:]

    return country_samples

In [ ]:
def train_model(training_model, X_context, y, epochs=1, verbose=1):
    history = training_model.fit([X_context], 
                                 [y],
                                 epochs=epochs,
                                 batch_size=32,   
                                 validation_split=0.1,
                                 verbose=verbose)
    return history

def permute_data(X_context, y, seed=10):
    np.random.seed(seed)
    p = np.random.permutation(y.shape[0])
    X_context = X_context[p]
    y = y[p]

    return X_context, y

In [ ]:
def train(df):
    print("Creating numpy arrays for Keras for training...")
    # Se crea el diccionario de datos
    country_samples = create_country_samples(df)
    print("Numpy arrays created")

    # Se extraen los datos de entrenamiento
    all_X_context_list = country_samples['X_train_context']
    all_y_list = country_samples['y_train']
    X_context = np.array(all_X_context_list)
    y = np.array(all_y_list)
    
    models = []
    train_losses = []
    val_losses = []
    # Bucle que entrena tantas veces como NUM_TRIALS diferentes modelos durante 100
    # epochs y los evalua. El objetivo es extraer el mejor modelo entrenado
    for t in range(NUM_TRIALS):
        print('Trial', t)
        # Se barajan los datos
        X_context, y = permute_data(X_context, y, seed=t)
        # Se crea el modelo (2 copias, una se entrena y la otra la almacenamos)
        model, training_model = construct_model(nb_context=X_context.shape[-1],
                                lstm_size=LSTM_SIZE,
                                nb_lookback_days=NB_LOOKBACK_DAYS)
        # Se entrena el modelo durante 100 epochs
        history = train_model(training_model, X_context, y, epochs=100, verbose=0)
        # Mejor epoch
        top_epoch = np.argmin(history.history['val_loss'])
        # Valor de pérdida en train de la mejor epoch
        train_loss = history.history['loss'][top_epoch]
        # Valor de périda en validación de la mejor epoch
        val_loss = history.history['val_loss'][top_epoch]
        # Se almacenan los errores de train y val
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        # Se guarda la copia del modelo
        models.append(model)

        print('Train Loss:', train_loss)
        print('Val Loss:', val_loss)

    country_casess = []
    # Se predicen los errores de test
    for model in models: 
        country_cases = lstm_get_test_rollouts(model, df, country_samples)
        country_casess.append(country_cases)

    test_case_maes = []
    for m in range(len(models)):
        # Para cada modelo se computa el MAE mediante los datos reales y los predichos
        total_loss = 0
        true_cases = np.sum(np.array(df['daily_vaccinations'])[-NB_TEST_DAYS:])
        pred_cases = np.sum(country_casess[m][-NB_TEST_DAYS:])
        total_loss += np.abs(true_cases - pred_cases)
        test_case_maes.append(total_loss)

    # Se selecciona el modelo con mejor MAE
    best_mae = np.min(test_case_maes)
    best_model = models[np.argmin(test_case_maes)]
    print("Done")
    return best_model, best_mae

In [ ]:
def lstm_get_test_rollouts(model, df, country_samples):
    country_cases = {}
    
    # Se obtienen los datos de test y se llama al predict 
    X_test_context = country_samples['X_test_context']

    # Primer día (lote) a predecir
    initial_context_input = country_samples['X_test_context'][0]
    # Resultados verdaderos
    y_test = country_samples['y_test']
    nb_test_days = y_test.shape[0]

    # Se hace la predicción
    preds = lstm_roll_out_predictions(model, initial_context_input, nb_test_days)

    return preds

In [ ]:
def lstm_roll_out_predictions(model, initial_context_input, nb_test_days):
    pred_output = np.zeros(nb_test_days)
    context_input = np.expand_dims(np.copy(initial_context_input), axis=0)
    for d in range(nb_test_days):
        pred = model.predict([context_input])
        pred_output[d] = pred
        context_input[:, :-1] = context_input[:, 1:]
        context_input[:, -1] = pred
    return pred_output

In [ ]:
def launch_train(data):
    data_train = pd.DataFrame(data['daily_vaccinations'])

    # se escalan los datos
    scaler = MinMaxScaler(feature_range=(0, 2))
    data_train['daily_vaccinations'] = scaler.fit_transform(data_train)

    modelo, mae = train(data_train)

    print("MAE:", scaler.inverse_transform(mae.reshape(1,-1)))
    
    return modelo, mae, scaler

In [ ]:
modelo, mae, scaler = launch_train(data_spain)

In [ ]:
# Plot test prediction
test_data = create_country_samples(data_spain)
cases_pred = lstm_get_test_rollouts(modelo, data_spain, test_data)
r = scaler.inverse_transform(cases_pred.reshape(1,-1))

In [ ]:
plt.figure(figsize=(12, 8))
vacc_true = list(data_spain['daily_vaccinations'])
sns.lineplot(x=range(0, 17), y=vacc_true[148:], label='True')
sns.lineplot(x=range(0, 17), y=r[0], label='Pred')
plt.legend()
plt.title('Vaccinations Real vs Pred')
plt.xlabel('Days')
plt.ylabel('Vaccinations')
plt.show(block=True)

### **Modelos alternativos a NN**

(Parte de Javier Galiana Romero. A partir de este punto se describe la parte realizada por el estudiante.Todo lo anterior fue hecho por Omar.)


#### 1) **Serie Temporal Modelo ARIMA**

In [ ]:
!pip install statsmodels

In [ ]:
data_vaccines = data_spain.copy()
data_vaccines = data_vaccines.set_index('date')
data_vaccines = data_vaccines['daily_vaccinations'].fillna(0)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

# daily_vaccinations[0] == nan
mod = ARIMA(data_vaccines[:152], order=(5,0,1))

model_fit = mod.fit()
model_fit.summary()

In [ ]:
forecast=model_fit.predict(start='2021-06-04', end='2021-06-16')
forecast.plot()

#### 2) **KNeighborsClassifier**


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold

In [ ]:
# Datos de España -> Dataframe 'data'
country_samples = create_country_samples(data)
X_train = country_samples['X_train_context']
Y_train = country_samples['y_train']


X_test = country_samples['X_test_context']
Y_test = country_samples['y_test']

nsamples, nx, ny = X_test.shape
X_test_2d = X_test.reshape((nsamples,nx*ny))

In [ ]:
#Escalamos los datos
#scaler = MinMaxScaler()

#aux = pd.DataFrame(scaler.fit_transform(aux), columns=aux.columns)

#Hacemos copia auxiliar para no tener que leer todo el tiempo el mismo conjunto de datos
#aux = country_samples['X_context'].copy()
aux = X_train.copy()



#Preparamos las etiquetas y las eliminamos 
#labels = country_samples['y'].copy()
labels = Y_train.copy()
#aux.drop(['price'], axis=1, inplace=True)


#Declaramos cross validation
cv = KFold(n_splits=5) 
print(cv)
#Para cada fold
#Variable acumuladora

mean_score = 0.0
for train_index, test_index in cv.split(aux):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #Seleccionamos los datos
    x_train, y_train = aux[train_index], labels[train_index]
    x_test, y_test = aux[test_index], labels[test_index]
    
    # https://stackoverflow.com/questions/34972142/sklearn-logistic-regression-valueerror-found-array-with-dim-3-estimator-expec
    nsamples, nx, ny = x_train.shape
    x_train = x_train.reshape((nsamples,nx*ny))
    
    nsamples, nx, ny = x_test.shape
    x_test = x_test.reshape((nsamples,nx*ny))

    #X_train = X_train.reshape(-1,27)
    #X_test = X_test.reshape(-1,27)
    #Declaramos modelo
    model = KNeighborsRegressor()
    model.fit(x_train, y_train)
    mean_score = mean_score + model.score(x_test,y_test)

#Obtenemos la media de los resultados
resultado = np.round(mean_score/5,2)
print("R2 con Cross validation: ", resultado) 

In [ ]:
r_KNNRegressor = scaler.inverse_transform(model.predict(X_test_2d).reshape(1,-1))

In [ ]:
from matplotlib import pylab

New_cases = list(data_spain['daily_vaccinations'])
pylab.plot(New_cases[151:], label='True')
pylab.plot(r_KNNRegressor[0], label='Pred')
pylab.legend()
pylab.title('Vaccinations Real vs Pred')
pylab.xlabel('Days')
pylab.ylabel('Vaccinations')
pylab.show(block=True)

#### 3) **LINEARSVR con Cross-Validation** 


In [ ]:
# PARTE DE LINEARSVR

#Import for Linear SVR
from sklearn.svm import LinearSVR
#import for apply crossvalidation
from sklearn.model_selection import KFold 

#Declaramos cross validation
cv = KFold(n_splits=5) 
print(cv)
#Para cada fold
#Variable acumuladora
mean_score = 0.0
for train_index, test_index in cv.split(aux):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #Seleccionamos los datos
    x_train, y_train = aux[train_index], labels[train_index]
    x_test, y_test = aux[test_index], labels[test_index]
    
    # https://stackoverflow.com/questions/34972142/sklearn-logistic-regression-valueerror-found-array-with-dim-3-estimator-expec
    nsamples, nx, ny = x_train.shape
    x_train = x_train.reshape((nsamples,nx*ny))
    
    nsamples, nx, ny = x_test.shape
    x_test = x_test.reshape((nsamples,nx*ny))
    
    #Declaramos modelo
    model = LinearSVR(max_iter=1000)
    model.fit(x_train, y_train)
    mean_score = mean_score + model.score(x_test,y_test)
    print("Valor R2: ", model.score(x_test,y_test))

#Obtenemos la media de los resultados
resultado = np.round(mean_score/5,2)
print("Resultado con Cross validation: ", resultado)

In [ ]:
r_linearsvr = scaler.inverse_transform(model.predict(X_test_2d).reshape(1,-1))


In [ ]:
from matplotlib import pylab

New_cases = list(data_spain['daily_vaccinations'])
pylab.plot(New_cases[151:], label='True')
pylab.plot(r_linearsvr[0], label='Pred')
pylab.legend()
pylab.title('Vaccinations Real vs Pred')
pylab.xlabel('Days')
pylab.ylabel('Vaccinations')
pylab.show(block=True)

#### 4) **LinearRegression con Cross-Validation**

In [ ]:
# PARTE DE LINEARREGRESSION

#Import for LinearRegression
from sklearn.linear_model import LinearRegression
#import for apply crossvalidation
from sklearn.model_selection import KFold 

#Declaramos cross validation
cv = KFold(n_splits=5) 
print(cv)
#Para cada fold
#Variable acumuladora
mean_score = 0.0
for train_index, test_index in cv.split(aux):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #Seleccionamos los datos
    x_train, y_train = aux[train_index], labels[train_index]
    x_test, y_test = aux[test_index], labels[test_index]
    
    # https://stackoverflow.com/questions/34972142/sklearn-logistic-regression-valueerror-found-array-with-dim-3-estimator-expec
    nsamples, nx, ny = x_train.shape
    x_train = x_train.reshape((nsamples,nx*ny))
    
    nsamples, nx, ny = x_test.shape
    x_test = x_test.reshape((nsamples,nx*ny))

    #Declaramos modelo
    model = LinearRegression()
    model.fit(x_train, y_train)
    mean_score = mean_score + model.score(x_test,y_test)
    print("Valor R2: ", model.score(x_test,y_test))

#Obtenemos la media de los resultados
resultado = np.round(mean_score/5,2)
print("Resultado con Cross validation: ", resultado)

In [ ]:
r_linearReg = scaler.inverse_transform(model.predict(X_test_2d).reshape(1,-1))

In [ ]:
from matplotlib import pylab

New_cases = list(data_spain['daily_vaccinations'])
pylab.plot(New_cases[151:], label='True')
pylab.plot(r_linearReg[0], label='Pred')
pylab.legend()
pylab.title('Vaccinations Real vs Pred')
pylab.xlabel('Days')
pylab.ylabel('Vaccinations')
pylab.show(block=True)

#### 5) **SVR**

In [ ]:
# PARTE DE SVR

#Import for Linear SVR
from sklearn.svm import SVR
#import for apply crossvalidation
from sklearn.model_selection import KFold 

#Declaramos cross validation
cv = KFold(n_splits=5) 
print(cv)
#Para cada fold
#Variable acumuladora
mean_score = 0.0
for train_index, test_index in cv.split(aux):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #Seleccionamos los datos
    x_train, y_train = aux[train_index], labels[train_index]
    x_test, y_test = aux[test_index], labels[test_index]
    
    # https://stackoverflow.com/questions/34972142/sklearn-logistic-regression-valueerror-found-array-with-dim-3-estimator-expec
    nsamples, nx, ny = x_train.shape
    x_train = x_train.reshape((nsamples,nx*ny))
    
    nsamples, nx, ny = x_test.shape
    x_test = x_test.reshape((nsamples,nx*ny))


    #Declaramos modelo
    # El mejor resultado se obtiene con el kernel polinómico, debido a que los datos
    # siguen un distribución polinomial perfecta de grado 2
    model = SVR(max_iter=1000, C=50, kernel="poly", coef0=2)
    model.fit(x_train, y_train)
    mean_score = mean_score + model.score(x_test,y_test)
    print("Valor R2: ", model.score(x_test,y_test))

#Obtenemos la media de los resultados
resultado = np.round(mean_score/5,2)
print("Resultado con Cross validation: ", resultado)

In [ ]:
r_svr= scaler.inverse_transform(model.predict(X_test_2d).reshape(1,-1))


In [ ]:
from matplotlib import pylab

New_cases = list(data_spain['daily_vaccinations'])
pylab.plot(New_cases[151:], label='True')
pylab.plot(r_svr[0], label='Pred')
pylab.legend()
pylab.title('Vaccinations Real vs Pred')
pylab.xlabel('Days')
pylab.ylabel('Vaccinations')
pylab.show(block=True)

#### 6) **RandomForestRegressor**

In [ ]:
# PARTE DE RANFOMFORESTREGRESSOR

#Import for RandomForest
from sklearn.ensemble import RandomForestRegressor
#import for apply crossvalidation
from sklearn.model_selection import KFold 


#Declaramos cross validation
cv = KFold(n_splits=5) 
print(cv)
#Para cada fold
#Variable acumuladora
mean_score = 0.0
for train_index, test_index in cv.split(aux):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #Seleccionamos los datos
    x_train, y_train = aux[train_index], labels[train_index]
    x_test, y_test = aux[test_index], labels[test_index]
    
    # https://stackoverflow.com/questions/34972142/sklearn-logistic-regression-valueerror-found-array-with-dim-3-estimator-expec
    nsamples, nx, ny = x_train.shape
    x_train = x_train.reshape((nsamples,nx*ny))
    
    nsamples, nx, ny = x_test.shape
    x_test = x_test.reshape((nsamples,nx*ny))
    #Declaramos modelo
    # Debido a que el espacio es monodimensional, la mayoría de los parámetros
    # de RF no son aplicables
    model = RandomForestRegressor(criterion="mse", bootstrap=True)
    model.fit(x_train, y_train)
    mean_score = mean_score + model.score(x_test,y_test)
    print("Valor R2: ", model.score(x_test,y_test))

#Obtenemos la media de los resultados
resultado = np.round(mean_score/5,2)
print("Resultado con Cross validation: ", resultado)

In [ ]:
r_randomFR = scaler.inverse_transform(model.predict(X_test_2d).reshape(-1,1))


In [ ]:
from matplotlib import pylab

New_cases = list(data_spain['daily_vaccinations'])
pylab.plot(New_cases[151:], label='True')
pylab.plot(r_randomFR, label='Pred')
pylab.legend()
pylab.title('Vaccinations Real vs Pred')
pylab.xlabel('Days')
pylab.ylabel('Vaccinations')
pylab.show(block=True)

## Clasificación

Aquí se trata de clasificar la variable Income Group para ver si puede discriminar a partir del número de vacunas el tipo de país (según nivel de produccion de servicios)

In [ ]:
df_ESG = pd.read_csv('../input/environment-social-and-governance-data/ESGData.csv')
df_table_description = pd.read_csv('../input/environment-social-and-governance-data/ESGCountry.csv')

# Obtención de las tuplas según sean País, Continente, etc.

# Etiqueta de países.
country_tags = df_table_description['Region'].copy()
mask_country_tag = country_tags.notna()
country_code = df_table_description.loc[mask_country_tag,'Country Code'].unique()

# Dataset con los datos de los países.
mask_country = df_ESG['Country Code'].isin(country_code)
df_country = df_ESG.loc[mask_country].copy()
df_country.reset_index(inplace=True)

# Etiqueta agregados
special_notes = df_table_description['Special Notes'].copy()
mask_region_tag = special_notes.str.extract(r'(aggregate)',expand=False).notna()
region_code = df_table_description.loc[mask_region_tag,'Country Code'].unique()

# dataset con los datos de los agregados
mask_region = df_ESG['Country Code'].isin(region_code)
df_region = df_ESG.loc[mask_region].copy()

# Máscara de tuplas que no son países
other_code = df_table_description.loc[~mask_country_tag & ~mask_region_tag,'Country Code'].unique()

# dataset con los datos que no son ni regiones ni países
mask_other = df_ESG['Country Code'].isin(other_code)
df_other = df_ESG.loc[mask_other].copy()

df_other['Country Name'].unique()

# pivotamos los datasets
col_year = df_country.filter(regex =("19*|20*")).columns.array
df_country = df_country.pivot(index=['Country Name','Country Code'],columns=['Indicator Name'],values=col_year).stack(0).reset_index()


In [ ]:
# Se cargan los datos
data = pd.read_csv("../input/d/gpreda/covid-world-vaccination-progress/country_vaccinations.csv")

In [ ]:
# Se declaran las columnas de interés
columns = ['country', 'date', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 
           'daily_vaccinations', 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 
           'people_fully_vaccinated_per_hundred', 'vaccines']

# Se seleccionan las columnas y se guarda el nuevo dataframe
data = data[columns]

# Se carga el segundo conjunto de datos
population_data = pd.read_csv("../input/populations/NamesPopulations.csv")

# Se renombra la columna GeoID para que coincida con el otro dataframe

population_data.rename(columns = {'GeoID': 'country'}, inplace = True)

data['date'] = pd.to_datetime(data['date'])

# Clip outliers

MIN_VALUE = 0.
MAX_VALUE = 100.
data[['total_vaccinations_per_hundred',
      'people_vaccinated_per_hundred', 
      'people_fully_vaccinated_per_hundred']] = np.clip(data[['total_vaccinations_per_hundred',
                                                              'people_vaccinated_per_hundred', 
                                                              'people_fully_vaccinated_per_hundred']],
                                                        MIN_VALUE, 
                                                        MAX_VALUE)

In [ ]:
data['numDays'] = (data.date-data.date.min()).astype('timedelta64[D]') + 1

In [ ]:
tags = []
for name in data['country'].values:
    tag = df_table_description[df_table_description['Short Name'] == name]['Income Group'].values
    if len(tag)==1:
        tags.append(tag[0])
    else:
        tags.append(np.nan)

df_tags = pd.DataFrame(tags,columns=['Income Group'])

In [ ]:
# añadimos la columna 'income group' al DataFrame 'data'
df_classification = pd.concat([data,df_tags],axis=1)

### Clasificación _"Income Group"_

In [ ]:
# Clasificación: "Income Group"
# dimensiones x = {numDays,total_vaccinations,daily_vaccinations}
#             y = {'Income Group'}


y = 'Income Group'
y_cols = df_classification.columns.isin([y])
x_cols = df_classification.columns.isin(['numDays','total_vaccinations','people_vaccinated','daily_vaccinations'])
#x_cols = df_classification.columns.isin(['people_fully_vaccinated'])

x = df_classification.loc[:,x_cols].dropna(axis=0)
y = df_classification.loc[:,y_cols].dropna(axis=0)

#### **1.2.1. Regresión Logística**

In [ ]:
from sklearn.linear_model import LogisticRegression

# escalado de datos
scaler = MinMaxScaler()
x_scaled = pd.DataFrame(scaler.fit_transform(x),columns=x.columns)

print("Clases: {}".format(y["Income Group"].astype(str).unique()))

#Declaramos cross validation
nsplits=4
cv = KFold(n_splits=nsplits) 
print(cv)
#Para cada fold
#Variable acumuladora
mean_score = 0.0
for train_index, test_index in cv.split(x_scaled):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #Seleccionamos los datos
    x_train, y_train = x_scaled.iloc[train_index], y.iloc[train_index]
    x_test, y_test = x_scaled.iloc[test_index], y.iloc[test_index]

    #Declaramos modelo
    model = LogisticRegression(C=100)
    model.fit(x_train, y_train)
    mean_score = mean_score + model.score(x_test,y_test)
    print("Accuracy: ", model.score(x_test,y_test))

#Obtenemos la media de los resultados
resultado = np.round(mean_score/nsplits,2)
print("Resultado con Cross validation: ", resultado)  

### **Clasificación con SVC**

In [ ]:
from sklearn.model_selection import StratifiedKFold

y = 'Income Group'
data_cols = ['numDays','total_vaccinations','people_vaccinated','daily_vaccinations', 'Income Group']
y_cols = df_classification.columns.isin([y])
x_cols = df_classification.columns.isin(['numDays','total_vaccinations','people_vaccinated','daily_vaccinations'])


df_xy = df_classification.loc[:,data_cols].dropna(axis=0)
x = df_xy[['numDays','total_vaccinations','people_vaccinated','daily_vaccinations']]
y =df_xy['Income Group']

scaler = MinMaxScaler()
x_scaled = pd.DataFrame(scaler.fit_transform(x),columns=x.columns)

#labels = y['Income Group']

nsplits=4
#Declaramos cross validation
cv = StratifiedKFold(n_splits=nsplits) 
#Para cada fold
#Variables acumuladoras
mean_score = 0.0
iteration = 0
precision = 0
for train_index, test_index in cv.split(x_scaled,y):
    iteration = iteration + 1
    #print("TRAIN:", train_index, "TEST:", test_index)
    #Seleccionamos los datos
    x_train, y_train = x_scaled.iloc[train_index], labels.iloc[train_index]
    x_test, y_test = x_scaled.iloc[test_index], labels.iloc[test_index]

    #Declaramos modelo
    model = SVC(gamma="auto", C=10000)
    model.fit(x_train, y_train)
    mean_score = mean_score + model.score(x_test,y_test)
    print("Iteration ", iteration)
    print("Accuracy: ", model.score(x_test,y_test))

    # Calculamos precision en cada fold
    y_predicted = model.predict(x_test)
    precision = precision_score(y_test, y_predicted, average=None, labels=y.unique())
    print("Precission: ", precision)
    print("-----------------")

#Obtenemos la media de los resultados
resultado = np.round(mean_score/nsplits,2)
print("Accuracy con Cross validation: ", resultado)    